In [111]:
import autograd.numpy as np
from autograd import grad
from scipy.optimize import minimize_scalar
import scipy.linalg as la

In [112]:
class BFGSOutput:
    def __init__(self, x, fx, converged, iter):
        self.x = x
        self.fx = fx
        self.converged = converged
        self.iter = iter

    def add_history(self,history):
        self.history = history

In [113]:
def bfgs_step(f, df, x, search_direction, method = "Armijo", armijo_param = 0.5) -> float:
    """ 
    computes BFGS step size

    Parameters:
    - f: objective function
    - df: derivative of objective function
    - x: current iterate
    - method: "full", "exact" or "Armijo"
    - armijo_param: Armijo parameter. Only needed if method = "Armijo"

    returns:
    - alpha: step size
    """
    if method == "full":
        return 1
    
    elif method == "exact":
        line_search = lambda alpha: f(x + alpha*search_direction)
        return minimize_scalar(line_search).x

    elif method == "Armijo":
        line_search = lambda alpha: f(x + alpha*search_direction)
        alpha = 1
        while line_search(alpha) > f(x) + armijo_param * alpha * np.dot(df(x), search_direction):
            alpha *= 0.5
        return alpha

In [114]:
def bfgs(f, x0, method = "Armijo", armijo_param = 0.5, max_iter = 10, tol = 1e-6, save_history = False):

    """ 
    Performs the BFGS optimization algorithm

    Parameters:
    - f: Objective function
    - x0: Initial guess for x
    - method: Method used to calculate the step size. Either "full", "exact" or "Armaijo" (default)
    - armijo_param: Only needed if method="Armajio" (default: 0.5)
    - max_iter: maximum number of iterations
    - tol: Convergence tolerance
    - save_history: If True, iteration history is returned

    returns:
    BFGS output with attributes:
    - x: Calculated minimizer
    - fx: Minimal value
    - converged: If True, method converged
    - n_iter: Number of iterations
    - (optional) history
    """

    n = x0.size

    x = x0

    df = grad(f) # compute the gradient of f
    H_inv = np.eye(n) # initialize inverse Hessian

    converged = False
    n_iter = max_iter

    if save_history:
        history = []
    
    for iter in range(max_iter):

        if save_history:
            history.append(x)
        
        if la.norm(df(x)) < tol:
            n_iter = iter
            converged = True
            break
        
        search_direction = - H_inv @ df(x)
        step_size = bfgs_step(f, df, x, search_direction, method = method, armijo_param = armijo_param)

        #perform updates
        x_old = x

        x = x + step_size * search_direction

        p = x - x_old
        q = df(x) - df(x_old)
        rho = 1 / np.dot(p,q)

        if iter == 0:
            # rescale first Hessian
            H_inv *= np.dot(p,q) / np.dot(q,q)

        H_inv = (np.eye(n) - rho * p @ q.T) @ H_inv @ (np.eye(n) - rho * p @ q.T)

    out = BFGSOutput(x, f(x), converged, n_iter)
    if save_history:
        out.add_history(history)
    return out

In [115]:
def f(x):
    return x[0]**2 + x[1]** 2